In [1]:
!pip install statsmodels

     |████████████████████████████████| 10.8MB 185kB/s eta 0:00:01
     |████████████████████████████████| 235kB 83.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [5]:
data

,PRODUCT_ID,ORDER_DATE,ORDER_QTY
0,Product_001,2019-01-02,850
1,Product_004,2019-01-02,700
2,Product_004,2019-01-02,900
3,Product_004,2019-01-02,550
4,Product_004,2019-01-02,300
...,...,...,...
225558,Product_098,2024-08-31,200
225559,Product_100,2024-08-31,800
225560,Product_100,2024-08-31,550
225561,Product_100,2024-08-31,850


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PRODUCT_ID  225563 non-null  object
 1   ORDER_DATE  225563 non-null  object
 2   ORDER_QTY   225563 non-null  int16 
dtypes: int16(1), object(2)
memory usage: 3.9+ MB


In [8]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_DATE  225563 non-null  datetime64[ns]
 2   ORDER_QTY   225563 non-null  int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB


In [10]:
data

,PRODUCT_ID,ORDER_DATE,ORDER_QTY
0,Product_001,2019-01-02,850
1,Product_004,2019-01-02,700
2,Product_004,2019-01-02,900
3,Product_004,2019-01-02,550
4,Product_004,2019-01-02,300
...,...,...,...
225558,Product_098,2024-08-31,200
225559,Product_100,2024-08-31,800
225560,Product_100,2024-08-31,550
225561,Product_100,2024-08-31,850


In [11]:
data = data.set_index('ORDER_DATE')

In [12]:
data

,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,
2019-01-02,Product_001,850
2019-01-02,Product_004,700
2019-01-02,Product_004,900
2019-01-02,Product_004,550
2019-01-02,Product_004,300
...,...,...
2024-08-31,Product_098,200
2024-08-31,Product_100,800
2024-08-31,Product_100,550


In [13]:
data.sort_index(inplace=True)

In [14]:
data

,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,
2019-01-02,Product_001,850
2019-01-02,Product_004,700
2019-01-02,Product_004,900
2019-01-02,Product_004,550
2019-01-02,Product_004,300
...,...,...
2024-08-31,Product_098,200
2024-08-31,Product_100,800
2024-08-31,Product_100,550


In [15]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [38]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [16]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['product_id','order_date', 'test_forecast_orders','rse', 'rmse','r2_score'])

In [17]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['product_id','order_date', 'forecast_order_qty'])

In [18]:
grouped_data

In [44]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    import pandas as pd

    print(test_df)
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    
    date_list = test_df.index.to_list()
    
    
    
   # print(f"Order_date: {date_list}")    
    new_row = {
        'product_id': product_id,
        'order_date': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'product_id': product_id,
        'order_date': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

13530    537.353131
13531    549.804686
13532    538.368357
13533    553.529134
13534    525.990260
13535    517.589784
13536    522.642301
13537    534.106066
13538    528.043225
13539    522.492035
13540    528.589942
13541    545.686329
13542    538.876343
13543    535.807857
13544    528.729567
13545    542.038266
13546    533.816373
13547    525.655403
13548    530.563869
13549    535.998652
13550    520.129920
13551    533.268958
13552    534.916404
13553    532.993399
13554    538.846818
13555    536.220820
13556    529.018280
13557    542.379782
13558    533.607148
13559    525.439350
13560    530.351923
13561    535.958597
13562    520.369437
13563    532.975604
13564    534.749935
13565    533.369187
13566    538.861564
13567    536.222950
13568    529.023952
13569    542.383949
13570    533.627017
13571    525.459414
13572    530.371870
13573    535.973642
13574    520.376512
13575    532.997872
13576    534.768585
13577    533.372377
13578    538.875047
13579    536.236793


In [20]:
test_results

,product_id,order_date,test_forecast_orders,rse,rmse,r2_score
0,Product_001,13530,537.353131,0.0,290.185585,-0.000545
1,Product_001,13531,549.804686,0.0,290.185585,-0.000545
2,Product_001,13532,538.368357,0.0,290.185585,-0.000545
3,Product_001,13533,553.529134,0.0,290.185585,-0.000545
4,Product_001,13534,525.990260,0.0,290.185585,-0.000545
...,...,...,...,...,...,...
45125,Product_100,3886,476.050933,0.0,295.283780,-0.021838
45126,Product_100,3887,522.781958,0.0,295.283780,-0.021838
45127,Product_100,3888,475.125436,0.0,295.283780,-0.021838
45128,Product_100,3889,498.165467,0.0,295.283780,-0.021838


In [21]:
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45130 entries, 0 to 45129
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_id            45130 non-null  object 
 1   order_date            45130 non-null  object 
 2   test_forecast_orders  45130 non-null  float64
 3   rse                   45130 non-null  float64
 4   rmse                  45130 non-null  float64
 5   r2_score              45130 non-null  float64
dtypes: float64(4), object(2)
memory usage: 2.1+ MB


In [22]:
test_results['order_date'] = pd.to_datetime(test_results['order_date'])

In [23]:
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45130 entries, 0 to 45129
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   product_id            45130 non-null  object        
 1   order_date            45130 non-null  datetime64[ns]
 2   test_forecast_orders  45130 non-null  float64       
 3   rse                   45130 non-null  float64       
 4   rmse                  45130 non-null  float64       
 5   r2_score              45130 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 2.1+ MB


In [24]:
test_results

,product_id,order_date,test_forecast_orders,rse,rmse,r2_score
0,Product_001,1970-01-01 00:00:00.000013530,537.353131,0.0,290.185585,-0.000545
1,Product_001,1970-01-01 00:00:00.000013531,549.804686,0.0,290.185585,-0.000545
2,Product_001,1970-01-01 00:00:00.000013532,538.368357,0.0,290.185585,-0.000545
3,Product_001,1970-01-01 00:00:00.000013533,553.529134,0.0,290.185585,-0.000545
4,Product_001,1970-01-01 00:00:00.000013534,525.990260,0.0,290.185585,-0.000545
...,...,...,...,...,...,...
45125,Product_100,1970-01-01 00:00:00.000003886,476.050933,0.0,295.283780,-0.021838
45126,Product_100,1970-01-01 00:00:00.000003887,522.781958,0.0,295.283780,-0.021838
45127,Product_100,1970-01-01 00:00:00.000003888,475.125436,0.0,295.283780,-0.021838
45128,Product_100,1970-01-01 00:00:00.000003889,498.165467,0.0,295.283780,-0.021838


In [26]:
for product_id, group in grouped_data:
    test[product_id,group]=product_id,group

NameError: name 'test' is not defined

In [27]:
test[product_id,group]

NameError: name 'test' is not defined

In [28]:
test = df[['product_id','group']]

KeyError: "None of [Index(['product_id', 'group'], dtype='object')] are in the [columns]"

In [29]:
# Initialize an empty DataFrame for forecast results
test = pd.DataFrame(columns=['product_id','group'])

In [31]:
for product_id, group in grouped_data:
    test['product_id','group']=product_id,group

In [32]:
test

,product_id,group,"(product_id, group)"
0,NaN,NaN,Product_100
1,NaN,NaN,PRODUCT_ID ORDER_QTY ORDER_DATE ...


In [39]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]

In [40]:
train_size

3112

In [41]:
train_data

,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,
2019-01-05,Product_100,300
2019-01-05,Product_100,400
2019-01-05,Product_100,650
2019-01-05,Product_100,150
2019-01-05,Product_100,800
2019-01-06,Product_100,100
2019-01-06,Product_100,550
2019-01-06,Product_100,350
2019-01-06,Product_100,300


In [42]:
test_data

,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,
2023-06-27,Product_100,850
2023-06-27,Product_100,200
2023-06-27,Product_100,300
2023-06-27,Product_100,50
2023-06-28,Product_100,950
2023-06-28,Product_100,400
2023-06-28,Product_100,850
2023-06-28,Product_100,150
2023-06-28,Product_100,400
